In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
def preprocess(df):
  # remove missing values
  def remove_missing_values(df):
    df = df.dropna()
    return df
  
  # remove outliers in fare amount
  def remove_fare_amount_outliers(df, lower_bound, upper_bound):
    df = df[(df['fare_amount'] >= lower_bound) & (df['fare_amount'] <= upper_bound)]
    return df

  # replace outliers in passenger count with the mode
  def replace_passenger_count_outliers(df):
    mode = df['passenger_count'].mode()
    df.loc[df['passenger_count'] == 0, 'passenger_count'] = mode
    return df

  # remove outliers in lat ang long
  def remove_lat_long_outliers(df):
    # range of long
    nyc_min_longtitude = -74.05
    nyc_max_longitude = -73.75
    # range of lat
    nyc_min_latitude = 40.6
    nyc_max_latitude = 40.85
    # only consider locations within NYC
    for long in ['pickup_longitude', 'dropoff_longtide']:
      df = df[(df[long] > nyc_min_longitude) & (df[long] < nyc_max_longitude)]
    for lat in ['pickup_latitude', 'dropoff_latitude']:
      df = df[(df[lat] > nyc_min_latitude) & (df[lat] < nyc_max_latitude)]
    return df

  df = remove_missing_values(df)
  df = remove_fare_amount_outliers(df, lower_bound=0, upper_bound=100)
  df = replace_passenger_count_outliers(df)
  df = remove_lat_long_outliers(df)
  return df

In [0]:
def feature_engineer(df):

  def create_time_features(df):
    # Create new col for time
    df['year']=df['pickup_datetime'].dt.year # dt function
    df['month']=df['pickup_datetime'].dt.month
    df['day']=df['pickup_datetime'].dt.day
    df['day_of_week']=df['pickup_datetime'].dt.dayofweek
    df['hour']=df['pickup_datetime'].dt.hour
    return df

  def euc_distance(lat1, long1, lat2, long2):
    # Euclidean distance
    return (((lat1-lat2)**2 + (long1-long2)**2)**0.5)

  def create_pickup_dropoff_dist_features(df):
    # Calculate distance travelled
    df['distance'] = euc_distance(df['pickup_latitude'], df['pickup_longitude'], df['dropoff_latitude'], df['dropoff_longitude'])
    return df

  def create_airport_dist_features(df):
    airports = {'JFK_Airport': (-73.78, 40.643),
       'LaGuardia_Airport': (-73.87, 40.77),
       'Newark_Airport': (-74.18, 40.69)}
    for airport in airports:
      df['pickup_dist_' + airport] = euc_distance(df['pickup_latitude'], df['pickup_longitude'], airports[airport][1], airports[airport][0])
      df['dropoff_dist_' + airport] = euc_distance(df['dropoff_latitude'], df['dropoff_longitude'], airports[airport][1], airports[airport][0])
    return df

  df = create_time_features(df)
  df = create_pickup_dropoff_dist_features(df)
  df = create_airport_dist_features(df)
  df = df.drop(['key'], axis=1)
  return df